In [6]:
# import config file with username and password
import sys
sys.path.append("T:/BCB/")
import BISdbConfig as config

# pybis needs environment variables to connect to the database.
import os
os.environ['DB_USERNAME']=config.user
os.environ['DB_PASSWORD']=config.password
os.environ['MONGODB_SERVER']="54.91.95.139"
os.environ['DB_DATABASE']="bis"

from pybis import db
from IPython.display import display
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 100)
import us

bisDB = db.Db.connect_mongodb("bis")
esaWPSpecies = bisDB["FWS_Work_Plan_Species"]


## Species names
#for record in esaWPSpecies.find({"Synthesis.Unique Scientific Names.1":{"$exists":True}}):
#    print ("Submitted Name:", record["Submitted Data"]["Scientific Name"])
#    try:
#        print ("ECOS Name:", record["ECOS Scrape"]["Scientific Name"])
#        print ("ECOS TSN:", record["ECOS Scrape"]["TSN"])
#    except:
#        pass
#    try:
#        print ("TESS Name:", record["TESS"]["SCINAME"])
#    except:
#        pass
#    for itisRecord in record["ITIS"]:
#        print ("ITIS:", itisRecord["nameWInd"], itisRecord["usage"], itisRecord["tsn"])
#    print ("----------")
    
sp = "Ambystoma barbouri"


# Build dataframe to fill out
columns=["FWS_range(y/n)", "BISON_occurrences", "SGCN_2015(y/n)", 
         "SGCN_2005(y/n)", "GAP_habitat(%)", "status1(%)", "status2(%)", 
         "status3(%)", "status4(%)"]
df0 = pd.DataFrame(index=[str(x) for x in us.STATES], columns=columns, data=0)
df0.index.name="State"

##########################################################################  FWS
# Fill out ECOS column with whether or not the ECOS account lists the state
FWS = esaWPSpecies.find_one({"Submitted Data.Scientific Name": sp})['FWS Range']
FWS_states = FWS['US State List']
for x in FWS_states:
    for y in FWS['US States']:
        if x == list(y.keys())[0]:
            state_name = y[x]['name'].strip()
            df0.loc[state_name, 'FWS_range(y/n)'] = 1
    
########################################################################  BISON
# How many occurrences are in each state?
bison = esaWPSpecies.find_one({"Submitted Data.Scientific Name": sp})['BISON']
bison_states = bison["US State Occurrences"]
for state in bison_states:
    state_name = list(state.keys())[0]
    df0.loc[state_name, "BISON_occurrences"]=int(state[state_name])

#########################################################################  SGCN 
SGCN = esaWPSpecies.find_one({"Submitted Data.Scientific Name": sp})['SGCN']
for x in SGCN['State List Summary']:
    if list(x.keys())[0] == 2005:
        statess = x['2005']['States']
        for z in statess:
            df0.loc[z, 'SGCN_2005(y/n)'] = 1
    elif list(x.keys())[0] == 2015:
        statess = x['2015']['States']
        for z in statess:
            df0.loc[z, 'SGCN_2015(y/n)'] = 1  
print(df0)

env: MONGODB_SERVER=54.91.95.139
env: DB_DATABASE=bis
                         FWS_range(y/n)  BISON_occurrences  SGCN_2015(y/n)  SGCN_2005(y/n)  GAP_habitat(%)  status1(%)  status2(%)  status3(%)  status4(%)
State                                                                                                                                                     
Alabama                             1.0                4.0             0.0             0.0             0.0         0.0         0.0         0.0         0.0
Alaska                              0.0                0.0             0.0             0.0             0.0         0.0         0.0         0.0         0.0
Arizona                             0.0                1.0             0.0             0.0             0.0         0.0         0.0         0.0         0.0
Arkansas                            0.0               36.0             0.0             0.0             0.0         0.0         0.0         0.0         0.0
California      